# Fase 1.- Google Search Engine

*Creamos la cuenta y el buscador en GoogleCloud. Solo 100 API calls/dia. Se ha activado búsqueda segura*

In [1]:
from credentials import API_KEY, Search_Engine_ID

La siguientes fucniones hacen una query y exploran el contenido de los N primeros links de una busqueda y appendan la informacion relevante en un excel (incluyendo cierta metadata). Muy interesante para posterior scrapping

In [2]:
import requests
import pandas as pd
import re

def build_payload(API_KEY, cx, query, start=1, num=10, date_restrict='m1', **params):
    """
    Construye el payload necesario para realizar la solicitud a la API de Google Custom Search.

    Parameters:
    - api_key (str): La clave de API de Google Custom Search.
    - cx (str): El identificador de búsqueda personalizado.
    - query (str): La cadena de búsqueda.

    Returns:
    - dict: El payload para la solicitud.
    """
    payload = {
        'key': API_KEY,
        'q':query,
        'cx': cx,
        'num': num,
        'dateRestrict': date_restrict
    }
    payload.update(params)
    return payload

def make_request(url, payload):
    """
    Realiza la solicitud a la API y maneja posibles errores.

    Parameters:
    - url (str): La URL de la API.
    - payload (dict): El payload de la solicitud.

    Returns:
    - dict: La respuesta de la API en formato JSON.
    """
    try:
        response = requests.get(url, params=payload)
        response.raise_for_status()  # Verifica si hay errores en la respuesta HTTP
        return response.json()
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error de conexión: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Tiempo de espera agotado: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Error desconocido: {err}")

def clean_filename(filename):
    filename = re.sub(r'[\\/*?:<>|]', "", filename)
    return filename

def main(query, result_total=10):
    url = "https://www.googleapis.com/customsearch/v1"
    items =[]
    reminder = result_total
    if reminder > 0:
        pages = (result_total // 10) + 1
    else:
        pages = result_total // 10 
    
    for i in range(pages    ):
        if pages == i + 1 and reminder > 0:
            payload = build_payload(API_KEY=API_KEY, cx=Search_Engine_ID, query=query, start=(i+1)*10, num=reminder)
        else:
            payload = build_payload(API_KEY=API_KEY, cx=Search_Engine_ID, query=query, start=(i+1)*10)
        response = make_request(url=url, payload=payload)
        items.extend(response['items'])
    query_string_clean = clean_filename(query)
    df = pd.json_normalize(items)
    df.to_excel('Google Result_{0}.xlsx'.format(query_string_clean), index =False)

In [3]:
import requests
def build_payload(API_KEY, cx, query, **params):
    """
    Construye el payload necesario para realizar la solicitud a la API de Google Custom Search.

    Parameters:
    - api_key (str): La clave de API de Google Custom Search.
    - cx (str): El identificador de búsqueda personalizado.
    - query (str): La cadena de búsqueda.

    Returns:
    - dict: El payload para la solicitud.
    """
    payload = {
        'key': API_KEY,
        'q':query,
        'cx': cx,
    }
    payload.update(params)
    return payload

def get_google_summary(API_KEY, cx, query):
    """
    Obtiene el objeto 'resumen' de Google Search para una búsqueda dada.

    Parameters:
    - api_key (str): La clave de API de Google Custom Search.
    - cx (str): El identificador de búsqueda personalizado.
    - query (str): La cadena de búsqueda.

    Returns:
    - str: El texto resumen de la entradilla o un mensaje indicando que no hay resumen.
    """
    url = "https://www.googleapis.com/customsearch/v1"
    payload = build_payload(API_KEY= API_KEY, cx= cx, query=query)
    
    response_json = make_request(url, payload)

    if response_json and 'items' in response_json:
        # Verifica si hay un resumen disponible en los resultados de la búsqueda
        first_result = response_json['items'][0]
        if 'snippet' in first_result:
            return first_result['snippet']
    
    return 'Item not summarized by google search engine'

# # Uso de las funciones
# query = 'f1 2012'

# resumen = get_google_summary(API_KEY, Search_Engine_ID, query)
# print(resumen)


Comento las lineas resultado, pero basicamente lo que me esta devolviendo es un resumen de cada link (Snippet es un atributo ya parseado de google que resumen el contenido del link i-essimo). Es una opcion a considerear para la fase1-Scrapping:

- https://cloud.google.com/generative-ai-app-builder/docs/snippets

Parece que la funcion no eta devolviendo los links 'normales' al hacer una busqueda, sino que se esta redirigiendo a la categoria 'Comparation Sites'. No he podido obtener mucha info de esta categoria (es como la categoria imagenes, videos al hacer una busqueda). Parece retornar links que podrian resultar de interes/links promocionados (?). Desde luego no son las entradas que nos interesan

Voy a fraccionar la busqueda para inspeccionar mejor los resultados

In [6]:
query = 'f1 2012' # Para este caso esta comprobadoq ue si hay entradilla, quermos ver como parsear el texto resumen
url = "https://www.googleapis.com/customsearch/v1"
payload = build_payload(API_KEY= API_KEY, cx= Search_Engine_ID, query=query)

response_json = make_request(url, payload)

In [18]:
import json
print(response_json.keys())
print(response_json['items'][0])
#print(json.dumps(response_json, indent=2))

dict_keys(['kind', 'url', 'queries', 'context', 'searchInformation', 'items'])
{'kind': 'customsearch#result', 'title': 'Norris Mexico GP "masterpiece" reminded Stella of Alonso\'s F1 ...', 'htmlTitle': 'Norris Mexico GP &quot;masterpiece&quot; reminded Stella of Alonso&#39;s <b>F1</b> ...', 'link': 'https://us.motorsport.com/f1/news/norris-mexico-gp-masterpiece-reminded-stella-of-alonsos-f1-valencia-2012-win/10540384/', 'displayLink': 'us.motorsport.com', 'snippet': 'Oct 30, 2023 ... McLaren Formula 1 team principal Andrea Stella has compared Lando Norris\' "masterpiece" Mexico Grand Prix drive to Fernando Alonso\'s iconic 2012 win for Ferrari\xa0...', 'htmlSnippet': 'Oct 30, 2023 <b>...</b> McLaren <b>Formula 1</b> team principal Andrea Stella has compared Lando Norris&#39; &quot;masterpiece&quot; Mexico Grand Prix drive to Fernando Alonso&#39;s iconic <b>2012</b> win for Ferrari&nbsp;...', 'cacheId': 'NUMlVG_q-lEJ', 'formattedUrl': 'https://us.motorsport.com/f1/news/norris-mexico...

Confirmamos que Snippet ofrece un resumen de cada link. Manualmente al hacer la busqueda sore 'f1 2012' no me aparecen como primeros links los de este resultado. Falta profundizar el porque. Vamos a probar a acotar mas los parametros de busqueda y a scrapear el html, ya que parece que el texto resumen (muchas veces ligado al resuemn de wikipedia) que ofrece google es un header 3 (h3) con el nombre de 'Descripcion'

In [6]:
def build_payload(api_key, cx, query, language='en'):
    payload = {
        'key': api_key,
        'cx': cx,
        'q': query,
        'lr': f'lang_{language}',  # Filtro para buscar solo en el idioma especificado
        'fileType': 'html',  # Obtiene el contenido en formato HTML
    }
    return payload

In [7]:
def get_google_summary(api_key, cx, query):

    api_url = "https://www.googleapis.com/customsearch/v1"
    payload = build_payload(api_key, cx, query)
    
    response_json = make_request(api_url, payload)

    if response_json and 'items' in response_json:
        # Verifica si hay un resumen disponible en los resultados de la búsqueda
        first_result = response_json['items'][0]
        if 'snippet' in first_result:
            snippet_text = first_result['snippet']

            # Intenta extraer el texto descriptivo de Wikipedia si está presente
            start_index = snippet_text.find('<span class="description">')
            end_index = snippet_text.find('</span>', start_index)
            if start_index != -1 and end_index != -1:
                wikipedia_description = snippet_text[start_index + len('<span class="description">'):end_index]
                return wikipedia_description

    return 'Item not summarized by google search engine'

# Uso de las funciones
query = 'Max Verstappen'

resumen = get_google_summary(api_key=API_KEY, cx=Search_Engine_ID, query=query)
print(resumen)

Item not summarized by google search engine


In [35]:
def build_payload(api_key, cx, query):
    payload = {
        'key': api_key,
        'cx': cx,
        'q': query,
        'fileType': 'html'
    }
    return payload

In [36]:
url = "https://www.googleapis.com/customsearch/v1"
payload = build_payload(api_key= API_KEY, cx= Search_Engine_ID, query='Max Verstappen')

response_json = make_request(url, payload)

In [37]:
for i in range(0,10):
    if 'items' in response_json:
        print(response_json['items'][i])

{'kind': 'customsearch#result', 'title': 'Max Verstappen - F1 Driver for Red Bull Racing', 'htmlTitle': '<b>Max Verstappen</b> - F1 Driver for Red Bull Racing', 'link': 'https://www.formula1.com/en/drivers/max-verstappen.html', 'displayLink': 'www.formula1.com', 'snippet': "Formula 1's youngest ever competitor – and now its reigning, two-time world champion. Say hello to Max Verstappen.", 'htmlSnippet': 'Formula 1&#39;s youngest ever competitor – and now its reigning, two-time world champion. Say hello to <b>Max Verstappen</b>.', 'cacheId': '15G_j1uI8W4J', 'formattedUrl': 'https://www.formula1.com/en/drivers/max-verstappen.html', 'htmlFormattedUrl': 'https://www.formula1.com/en/drivers/<b>max</b>-<b>verstappen</b>.html', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcTYep9-kxs1FG_YjCNoLiMIIpdVi2f6IjoCthA6lGCzdRiodgQZ-2a9R3PV', 'width': '225', 'height': '225'}], 'website': [{'image': 'https://media.formula1.com/content/dam/fom-website/driver

In [17]:
import json
#print(response_json.keys())
#print(response_json['items'])
print(json.dumps(response_json['items'], indent=5))

[
     {
          "kind": "customsearch#result",
          "title": "Max Verstappen - F1 Driver for Red Bull Racing",
          "htmlTitle": "<b>Max Verstappen</b> - F1 Driver for Red Bull Racing",
          "link": "https://www.formula1.com/en/drivers/max-verstappen.html",
          "displayLink": "www.formula1.com",
          "snippet": "Formula 1's youngest ever competitor \u2013 and now its reigning, two-time world champion. Say hello to Max Verstappen.",
          "htmlSnippet": "Formula 1&#39;s youngest ever competitor \u2013 and now its reigning, two-time world champion. Say hello to <b>Max Verstappen</b>.",
          "cacheId": "15G_j1uI8W4J",
          "formattedUrl": "https://www.formula1.com/en/drivers/max-verstappen.html",
          "htmlFormattedUrl": "https://www.formula1.com/en/drivers/<b>max</b>-<b>verstappen</b>.html",
          "pagemap": {
               "cse_thumbnail": [
                    {
                         "src": "https://encrypted-tbn3.gstatic.com/imag